Here is the propose pipeline 
I aint gonna fine tune my own Bert for NER we will see that in the future but lets come forward
There is the pipeline
1. To extract the NER:
    - Test and write out all the hardcoded details of the NER that will be available
    - Create the embeddings for the documents using SentenceTransformer(a generalized one tho)
    - Next ask it the NER question and after asking pass the answer into the LLM which will then refine the output -> problem? Use COT maybe to make sure it does not cofuse the names or 
        dates
    - Display the result of the Pipeline 1 ->
2. Extracting the Red Flags
    - We will use ALeaseBert for this 
    - Get the top 3 answers for this 
    - Pass it into the LLM 
    - Get the refined response
    

In [5]:
from transformers import AlbertTokenizer,AlbertModel
import tqdm as notebook_tqdm
import numpy as np
# Specify the directory containing the model files
model_dir = r"C:\Users\SAR\Desktop\Hassan\Hackathon\models"

# Load the tokenizer
tokenizer = AlbertTokenizer.from_pretrained(model_dir)
model = AlbertModel.from_pretrained(model_dir)

print("Model and tokenizer loaded successfully!")


Model and tokenizer loaded successfully!


In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import faiss
import re
from docx import Document

def read_docx(file_path):
    """
    Extract text from a .docx file.
    """
    doc = Document(file_path)
    text = []
    for paragraph in doc.paragraphs:
        if paragraph.text.strip():  # Only include non-empty paragraphs
            text.append(paragraph.text.strip())
    return "\n".join(text)

def clean_text(text):
    """
    Clean text for better embedding generation.
    """
    # Remove unnecessary whitespace
    text = re.sub(r'\s+', ' ', text)
    # Remove non-ASCII characters
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    return text.strip()

raw_text = read_docx("propositions_output.docx")
cleaned_text = clean_text(raw_text)

# Initialize the RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=250,     # Maximum chunk size (in characters)
    chunk_overlap=20,   # Overlap between chunks (in characters)
    separators=["\n\n", "\n", ".", " ", ""],  # Prioritized separators
)

# Split the lease text into chunks
chunks = text_splitter.split_text(cleaned_text)

# Print the chunks
print("Text Chunks:")
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1}:")
    print(chunk)
    print("---")
    


Text Chunks:
Chunk 1:
Propositions Output [ "EX1A-6 MAT CTRCT 11 ark7_ex6-10.htm EXHIBIT 6.10 Exhibit 6.10 RESIDENTIAL LEASE AGREEMENT This Lease Agreement is made and entered on [CONTRACT_DATE].", "The Effective Date of the Lease Agreement is [CONTRACT_DATE]
---
Chunk 2:
.", "ARK7 PROPERTIES LLC is the Landlord.", "TENANT1 is the Tenant.", "TENANT2 is the Tenant.", "The parties agree to pay rent using personal check, money order, or cashier s check.", "Make your check payable to ARK7 INC
---
Chunk 3:
.", "Mail your check to the company address listed below.", "Pay your rent before the due date each month.", "The address is 535 Mission St, 14th Floor, San Francisco, CA 94105
---
Chunk 4:
.", "If any payment is returned for non-sufficient funds or because Tenant stops payments, then Landlord may require Tenant to pay Rent in cash for three months
---
Chunk 5:
.", "After the return of a payment, Landlord may require Tenant to pay Rent by cashier s check or money order
---
Chunk 6:
.", "I

In [7]:
# Embedding function
def embed_clause(clause):
    inputs = tokenizer(clause, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    # Use pooler output for fixed-size embeddings
    return outputs.pooler_output.detach().numpy().flatten()

embeddings = np.array([embed_clause(chunk) for chunk in chunks], dtype="float32")

# Create FAISS index
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

print(f"Number of vectors in index: {index.ntotal}")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Number of vectors in index: 126


In [9]:
query = "What is the contract start date?"

# Step 1: Embed the query
query_embedding = embed_clause(query)

# Step 2: Reshape the query embedding for FAISS search
query_embedding = query_embedding.reshape(1, -1)

# Step 3: Perform the search in the FAISS index
D, I = index.search(query_embedding, k=20)  # Retrieve top 5 closest embeddings

# Step 4: Retrieve the top matching chunks
retrieved_chunks = [chunks[i] for i in I[0]]

# Step 5: Print the results
print("Query:", query)
print("\nTop Matching Chunks:")
for idx, chunk in enumerate(retrieved_chunks):
    print(f"{idx + 1}. {chunk} (Distance: {D[0][idx]:.2f})")
    print("---")


Query: What is the contract start date?

Top Matching Chunks:
1. .", "The landlord and tenant may change their addresses from time to time.", "The landlord and tenant must provide notice for changing their addresses (Distance: 0.12)
---
2. .", "Chunk 8 is about damages caused by the Tenant or by a guest or licensee of the Tenant.", "Chunk 8 also discusses cleaning the Premises, if necessary, upon termination of the tenancy (Distance: 0.13)
---
3. .", "Installation and maintenance of the Satellite Dish must not damage the Premises or its walls in any way.", "The Tenant remains strictly liable for any injury or damage to persons or property caused by the satellite dish (Distance: 0.13)
---
4. .", "The bi-monthly basis is every two months.", "The RUBs formula becomes obsolete or is invalidated in whole or in part (Distance: 0.16)
---
5. .", "If allowed, the Satellite Dish must be installed inside balcony railings or window.", "The Satellite Dish must be mounted so as to not be visible fro

In [11]:
pip install accelerate


Note: you may need to restart the kernel to use updated packages.


In [5]:
# pip install accelerate
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large", device_map="auto", torch_dtype=torch.float16)

input_text = "translate English to German: How old are you?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)